# Analysis

Cross tabulations and generalized linear models using [Samplics](https://github.com/samplics-org/samplics).

In [34]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from typing import TypeAlias, Literal
from samplics.categorical import CrossTabulation
from samplics.regression.glm import SurveyGLM

gss: pd.DataFrame = pd.read_csv("data/Theo_extract_3-20-2023.csv")
gss_saf: pd.DataFrame = pd.read_csv("data/Safiya_extract_3-20-2023.csv")

# --------------------------------
# --- Quick clean up (ignore) ----
# --------------------------------
# Any number type (lazy)
Number: TypeAlias = np.number | float | pd.Int64Dtype


def recode_party(partyid: Number) -> Literal["Democrat", "Republican", "Other", pd.NA]:
    """Recode and collapse GSS' `partyid` from numbers to a few strings.

    Parameters
    ----------
    partyid : Number

    Return
    ------
    Literal["Democrat", "Republican", "Other", pd.NA]
    """

    match partyid:
        case _ if pd.isna(partyid):
            return pd.NA
        case 0 | 1 | 2:
            return "Democrat"
        case 4 | 5 | 6:
            return "Republican"
        case 3 | 7:
            return "Other"
        case _:
            return pd.NA


def recode_degree(degree: Number) -> Literal["No degree", "HS or assoc", "College"]:
    """Recode and collapse degree into strings with less categories.

    Parameters
    ----------
    degree : Number

    Return
    ------
    Literal["No degree", "HS or assoc", "College"]
    """

    match degree:
        case _ if pd.isna(degree):
            return pd.NA
        case 0:
            return "No degree"
        case 1 | 2:
            return "HS or assoc"
        case 3 | 4:
            return "College"
        case _:
            return pd.NA


# R's political party
gss_saf["partyid"] = gss_saf["partyid"].map(recode_party).astype("category")

# Obvious features
# gss_saf["age"] = gss_saf["age"].astype("Int64")
gss_saf["degree"] = (
    gss_saf["degree"].astype("Int64").map(recode_degree).astype("category")
)
gss_saf["sex"] = (
    gss_saf["sex"].astype("Int64").map({1: "Male", 2: "Female"}).astype("category")
)

# Does R speak a language other than English or Spanish?
gss_saf["othlang"] = (
    gss_saf["othlang"].astype("Int64").map({1: "Yes", 2: "No"}).astype("category")
)

# Immigration should be decreased (recoded from letin1a)
gss_saf["decrease_imm"] = (
    gss_saf["decrease_imm"].astype("Int64").map({1: "Yes", 0: "No"}).astype("category")
)

# How important is it for respondent's children to be able to think for themselves?
gss["thnkself"] = (
    gss["thnkself"]
    .astype("Int64")
    .map({1: "Most", 2: "Second", 3: "Third", 4: "Fourth", 5: "Last"})
    .astype("category")
)

# Does the respondent believe in life after death?
gss["postlife"] = gss["postlife"].map({1: "Yes", 2: "No"})


## Cross tabulations

In [35]:
tab_thnk_post = CrossTabulation("proportion")
tab_thnk_post.tabulate(
    vars=gss[["postlife", "thnkself"]],
    samp_weight=gss["wtssall"],
    # There isn't any stratification for the variables used
    # ...or there's only one PSU. Something like that.
    # stratum=gss["vstrat"],
    psu=gss["vpsu"],
    remove_nan=True,
)

print(tab_thnk_post)



Cross-tabulation of postlife and thnkself
 Number of strata: 1
 Number of PSUs: 2
 Number of observations: 8167
 Degrees of freedom: 1.00

 postlife thnkself  proportion  stderror  lower_ci  upper_ci
      No   Fourth    0.030365  0.002184  0.012053  0.074401
      No     Last    0.010181  0.000394  0.006219  0.016625
      No     Most    0.092465  0.001289  0.077340  0.110196
      No   Second    0.034407  0.000216  0.031767  0.037258
      No    Third    0.031475  0.000818  0.022583  0.043711
     Yes   Fourth    0.140978  0.009139  0.059188  0.299778
     Yes     Last    0.037669  0.000800  0.028721  0.049265
     Yes     Most    0.326757  0.003157  0.287977  0.368060
     Yes   Second    0.157055  0.000825  0.146851  0.167829
     Yes    Third    0.138646  0.002275  0.112183  0.170156

Pearson (with Rao-Scott adjustment):
	Unadjusted - chi2(4): 19.9821 with p-value of 0.0005
	Adjusted - F(1.00, 1.00): 6.1194  with p-value of 0.2446

  Likelihood ratio (with Rao-Scott adjustment):


## Generalized linear regression

In [36]:
# Explicitly drop NAs because remove_nan doesn't do it for some reason
gss_saf = gss_saf.dropna()

# Variable names for features
X_features = ["year", "age", "degree", "sex", "partyid", "coninc"]
X_imm_dummies = pd.get_dummies(gss_saf[X_features], drop_first=True)
X_imm_dummies = sm.add_constant(X_imm_dummies)

# Target feature
y_feature = pd.get_dummies(gss_saf["decrease_imm"])

# I wish samplics wasn't incomplete.
# Fit the model
glm_imm = SurveyGLM()
glm_imm.estimate(
    y=y_feature,
    x=X_imm_dummies,
    samp_weight=gss_saf["wtssnrps"],
    # stratum=saf_gss["vstrat"],
    psu=gss_saf["vpsu"],
    remove_nan=True,
)

results_imm = pd.DataFrame(
    [glm.beta, *glm.cov_beta], columns=X_features, index=["coef", *X_features]
).T

print(results_imm)


ValueError: operands could not be broadcast together with shapes (9045,2) (9045,) 

In [37]:
# Binomial regression
glm_imm = sm.GLM(
    y_feature,
    X_imm_dummies,
    family=sm.families.Binomial(),
    var_weights=gss_saf["wtssnrps"],
    missing="drop",
)

results_imm = glm_imm.fit()
print(results_imm.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:          ['No', 'Yes']   No. Observations:                 9045
Model:                            GLM   Df Residuals:                     9036
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5462.6
Date:                Fri, 24 Mar 2023   Deviance:                       10925.
Time:                        01:09:49   Pearson chi2:                 8.77e+03
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1048
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -160.0024     12